In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV, KFold, cross_validate
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import cross_decomposition
import plotly.express as px 

### Constants

In [2]:
data_dir = "../../data/processed/"
#Seed for controlling any random procedure during the experiments
seed = 99

### Creating estimators

In [3]:
least_squares = linear_model.LinearRegression()
ridge = linear_model.Ridge(random_state=seed)
lasso = linear_model.Lasso()
pls = cross_decomposition.PLSRegression()
svr = svm.SVR()

### Defining hyperparameter grids
Each algorithm has its corresponding hyperparameter grid for later use in grid search inner cross-validation loop.

In [13]:
np.linspace(start = 2, stop=25, num=24).astype(int)

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25])

In [14]:
least_squares_grid = {} #Ordinary least square doesn't have hyperparamters

ridge_grid = {'alpha' : np.logspace(-6, 6, 13)} #Alpha between 1.e-06 and 1.e+06
lasso_grid = {'alpha' : np.logspace(-6, 6, 13)} 
pls_grid = {'n_components': np.linspace(start = 2, stop=25, num=24).astype(int)} #Between 2 and 25 Principal Components

svr_grid = [
    #Grid for linear kernel
    {'C': np.logspace(-6, 6, 13), 'kernel': ['linear']},
    #Grid for rbf and sigmoid kernel
    {'C': np.logspace(-6, 6, 13), 'gamma': np.logspace(-6, 6, 13), 'kernel': ['rbf', 'sigmoid']},
    #Grid for polinomial kernel
    {'C': np.logspace(-6, 6, 13), 'gamma': np.logspace(-6, 6, 13), 'kernel': ['poly'], 'degree': [2,3,4]}
 ]


### Mapping each estimator with its hyperparameter grid

In [15]:
learning_methods = [{'name': 'OLS', 'estimator':least_squares, 'hyperparameter_grid': least_squares_grid},
                    {'name': 'RIDGE','estimator':ridge, 'hyperparameter_grid': ridge_grid},
                    {'name': 'LASSO', 'estimator':lasso, 'hyperparameter_grid': lasso_grid},
                    {'name': 'SVR', 'estimator':svr, 'hyperparameter_grid': svr_grid},
                    {'name': 'PLS', 'estimator':pls, 'hyperparameter_grid': pls_grid}]

### Getting data ready

In [16]:
def datasets():
    
    #TODO: This function should change when working in Google Collabs
    datasets_names = ['A.csv', 'B.csv', 'C.csv', 'D.csv']
    
    for dataset_name in datasets_names:
        protein_dataset = pd.read_csv(data_dir + dataset_name)
        proteins_X = protein_dataset.iloc[:, 1:].to_numpy()
        proteins_X = preprocessing.scale(proteins_X) #Z-Score standarization
        proteins_y = protein_dataset.iloc[:,0].to_numpy()
        
        yield {'name': dataset_name, 'features': proteins_X, 'target': proteins_y}

### Training and validation

In [17]:
#The nested_cv function returns a dictionary like with the outer cross-validation loop scores for every learning method.
#The dictionary looks like:
#{
#    'r2': {'PLS': [0.99,...,0.67], 'SVR': [0.94,..., 0.98], ... , 'OLS': [0.4, ..., 0.32]}, 
#  'rmse': {'PLS': [1.297116,...,2.297116], 'SVR': [1.291,..., 0.29471], ... , 'OLS': [3.19283, ..., 5.827391]}
#}
def nested_cv(features, target):
    
    #Score metric used for hyperparameter optimization in inner CV loop
    inner_scoring = 'r2'
    
    #Score metrics used in outer CV loop for generalization performance estimation of the learning method 
    outer_scoring = ['r2', 'neg_root_mean_squared_error']
    
    outer_scores_r2 = {}
    outer_scores_rmse = {}
    
    for learning_method in learning_methods:
        inner_cv = KFold(n_splits=10, shuffle=True, random_state=seed)
        outer_cv = KFold(n_splits=10, shuffle=True, random_state=seed)

        #Inner CV estimator
        est = GridSearchCV(estimator = learning_method['estimator'], 
                           param_grid = learning_method['hyperparameter_grid'], 
                           cv = inner_cv, 
                           scoring = inner_scoring)

        #For each train-test dataset split in the outer cross validation loop, 
        #cross_validate function will call the fit() method of the GridSearchCV estimator.
        outer_cv_results = cross_validate(estimator = est, 
                                          X = features, y = target, 
                                          cv = outer_cv, scoring = outer_scoring)
        
        outer_scores_r2[learning_method['name']] = outer_cv_results['test_r2']
        
        #Inside CV, the RSME score is managed as a negative RMSE. Multiplying it by -1 will turn it into the usual positive RMSE  
        outer_scores_rmse[learning_method['name']] = outer_cv_results['test_neg_root_mean_squared_error']*-1 
       
    return {'r2': outer_scores_r2, 'rmse':outer_scores_rmse}


### Running experiments

In [18]:
#For each dataset (A,B,C,D)
for dataset in datasets():
    outer_scores = nested_cv(dataset['features'], dataset['target'])
    
    outer_scores_df = pd.DataFrame(outer_scores['r2'])
    #Export scores to Latex table
    outer_scores_df.to_latex(buf='../modeling/RSquared_r2_scores_' + dataset['name'] + '.txt', index_names= False,
                            caption="RSquared scores for dataset" + dataset['name'])
    
    outer_scores_df = pd.DataFrame(outer_scores['rmse'])
    outer_scores_df.to_latex(buf='../modeling/RSquared_rmse_scores_' + dataset['name'] + '.txt', index_names=False,
                            caption="RMSE scores for dataset" + dataset['name'])

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.8616939098845187, tolerance: 0.010078838787878788

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.9333324465572372, tolerance: 0.010723396363636364

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.3348794759340152, tolerance: 0.010017515151515152

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge.

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.4513747338735707, tolerance: 0.010078838787878788

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.4464691356783614, tolerance: 0.010723396363636364

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.3738774163569962, tolerance: 0.010017515151515152

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge.

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.3718216142900808, tolerance: 0.008898038787878789

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.967877624667179, tolerance: 0.010218242424242425

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.7179162518193865, tolerance: 0.009753105454545455

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. 

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.06303265210868148, tolerance: 0.008898038787878789

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.13226275777598673, tolerance: 0.010218242424242425

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.15858248758937776, tolerance: 0.010084721176470588

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not conver

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.13547686302742257, tolerance: 0.010429562352941176

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.010671504655903163, tolerance: 0.010002382352941177

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.026770878466429338, tolerance: 0.00927928

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You 

...

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.890897430377578, tolerance: 0.02788192511627907

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.134275334759266, tolerance: 0.024031859999999995

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.593050056450203, tolerance: 0.026879227441860474

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 10.573315279644593, tolerance: 0.02492573627906977

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.805562704086568, tolerance: 0.01691106976744186

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.975774214611596, tolerance: 0.024601596744186046

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 8.780126589443599, tolerance: 0.02377962744186047

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 7.929931512168543, tolerance: 0.024279376279069766

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 6.148899071752575, tolerance: 0.020968317674418598

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.20497441774170255, tolerance: 0.020968317674418598

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.27138776334479076, tolerance: 0.023528706511627914

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.15537374172426155, tolerance: 0.023544539069767438

c:\users\benjamin\.conda\envs\scikit-jupyter\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning:

Objective did not conver

KeyboardInterrupt: 